# Projet de Data Management : Gestion d'un fonds Multi-Asset

**Période de simulation :** 01-01-2023 – 31-12-2024  
**Auteurs :** SHILI Eya, CHERIFI Dhelia, BOUSSELMAME Adam

---

## Description

Ce notebook orchestre l'exécution du pipeline complet, qui comprend :

- **Création et peuplement de la base de données**
- **Acquisition et préparation des données**
- **Exécution des stratégies d'investissement**  
  *(Low Volatility, Low Turnover, Equity Only)*
- **Mise à jour des portefeuilles et enregistrement des deals**
- **Évaluation finale des performances**

---

*Ce notebook sert de point d'entrée pour lancer et suivre l'ensemble du processus de gestion du fonds multi-asset sur la période de simulation.*


In [1]:
import pandas as pd
from datetime import datetime, timedelta
import base_builder
import base_update
import performances
from tqdm.auto import tqdm
import strategies
import os 


# Suppression de la base de données si elle existe déjà
# pour éviter les conflits lors de la création
db_file = "fund_database.db"
if os.path.exists(db_file):
    os.remove(db_file)

def get_mondays_between(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    days_to_next_monday = (7 - start_date.weekday()) % 7
    next_monday = start_date if days_to_next_monday == 0 else start_date + timedelta(days=days_to_next_monday)
    mondays = []
    current = next_monday
    while current <= end_date:
        mondays.append(current.strftime("%Y-%m-%d"))
        current += timedelta(days=7)
    return mondays

def initialize_databases():
    print("=== Initialisation des bases de données ===")
    base_builder.create_tables()
    try:
        n_low_risk = int(input("Nombre de clients Low risk : "))
        n_low_turnover = int(input("Nombre de clients Low turnover : "))
        n_high_yield = int(input("Nombre de clients High yield equity only : "))
    except ValueError:
        print("Erreur : veuillez saisir des nombres entiers pour les clients.")
        return False
    base_builder.generate_clients(n_low_risk, n_low_turnover, n_high_yield)
    base_builder.generate_managers()
    base_builder.generate_portfolios(n_low_risk, n_low_turnover, n_high_yield)
    print("Bases de données initialisées.\n")
    import data_collector #Le module contient du code qui se lance automatiquement à l'import, il ne faut pas supprimer cet import
    return True


def main():
    nbtrees = 100   # Nombre d'arbres à utiliser pour la stratégie Equity Only
    start_date_str = "2023-01-01" #Date de "création" du fonds
    end_date_str = "2024-12-31" #Date de fin du fonds
    mondays = get_mondays_between(start_date_str, end_date_str)
    if len(mondays) < 2:
        print("La période définie ne permet pas de générer au moins une période hebdomadaire.")
        return
    if not initialize_databases():
        return
    
    perf_par_semaine = {}
    

    for i in tqdm(range(len(mondays)), desc="Traitement des périodes", unit="période"): # Lancement du traitement
        if i == 0:
            base_update.update_profiles_table(
                mondays[i],
                (datetime.strptime(mondays[i], "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"),
                nbtrees
            )
            #End_Date correspond à 1 jour avant le lundi du lancement des stratégies pour éviter le look-forward bias
            
        elif i < len(mondays) - 1:
            start_perf = mondays[i-1]
            end_perf = mondays[i]
            print(f"\nSemaine {i+1} : {mondays[i]} à {mondays[i+1]}")
            print(f"→ Calcul des performances de {start_perf} à {end_perf}")
            df_perf = performances.calculer_performance_semaine(start_perf, end_perf)
            key = f"{start_perf} to {end_perf}"
            perf_par_semaine[key] = df_perf
            base_update.update_profiles_table(
                mondays[i],
                (datetime.strptime(mondays[i], "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"),
                nbtrees
            ) 
            #End_Date correspond à 1 jour avant le lundi du lancement des stratégies pour éviter le look-forward bias
        else:
            start_perf = mondays[i-1]
            end_perf = mondays[i]
            print(f"Calcul des performances de {start_perf} à {end_perf}")
            print(f"\nDernière période : Calcul des performances de {start_perf} à {end_perf}")
            df_perf = performances.calculer_performance_semaine(start_perf, end_perf)
            key = f"{start_perf} to {end_perf}"
            perf_par_semaine[key] = df_perf

    if perf_par_semaine:
        df_total = pd.concat(perf_par_semaine.values(), keys=perf_par_semaine.keys())
        df_total.index.names = ["Période", "Index"]
        
        # Formatage en pourcentage pour les données hebdomadaires
        df_total_formatted = df_total.copy()
        percentage_cols = ['rendement', 'volatilite', 'alpha']
        for col in df_total_formatted.columns:
            if pd.api.types.is_numeric_dtype(df_total_formatted[col]) and col != 'portfolio_id':
                if col in percentage_cols:
                    df_total_formatted[col] = (df_total_formatted[col] * 100).map("{:.2f}%".format)
                else:
                    df_total_formatted[col] = df_total_formatted[col].map("{:.2f}".format)
            else:
                df_total_formatted[col] = df_total_formatted[col].astype(str)
        
        # Formatage en pourcentage pour l'agrégation globale par portefeuille
        aggregated_by_portfolio = performances.compute_aggregated_performance_by_portfolio(df_total)
        aggregated_by_portfolio_formatted = aggregated_by_portfolio.copy()
        percentage_cols_port = ['aggregated_return', 'aggregated_volatility', 'aggregated_alpha']
        for col in aggregated_by_portfolio_formatted.columns:
            if pd.api.types.is_numeric_dtype(aggregated_by_portfolio_formatted[col]) and col != 'profil':
                if col in percentage_cols_port:
                    aggregated_by_portfolio_formatted[col] = (aggregated_by_portfolio_formatted[col] * 100).map("{:.2f}%".format)
                else:
                    aggregated_by_portfolio_formatted[col] = aggregated_by_portfolio_formatted[col].map("{:.2f}".format)
            else:
                aggregated_by_portfolio_formatted[col] = aggregated_by_portfolio_formatted[col].astype(str)
        
        with pd.ExcelWriter("rapport_performances.xlsx", engine="xlsxwriter") as writer:
            # Export du DataFrame hebdomadaire avec l'index
            df_total_formatted.to_excel(writer, sheet_name="Performances Hebdomadaires", index=True)
            
            # Agrégation par portefeuille
            aggregated_by_portfolio = performances.compute_aggregated_performance_by_portfolio(df_total)
            # Formatage pour l'agrégation par portefeuille
            aggregated_by_portfolio_formatted = aggregated_by_portfolio.copy()
            percentage_cols_port = ['aggregated_return', 'aggregated_volatility', 'aggregated_alpha']
            for col in aggregated_by_portfolio_formatted.columns:
                if pd.api.types.is_numeric_dtype(aggregated_by_portfolio_formatted[col]) and col != 'profil':
                    if col in percentage_cols_port:
                        aggregated_by_portfolio_formatted[col] = (aggregated_by_portfolio_formatted[col] * 100).map("{:.2f}%".format)
                    else:
                        aggregated_by_portfolio_formatted[col] = aggregated_by_portfolio_formatted[col].map("{:.2f}".format)
                else:
                    aggregated_by_portfolio_formatted[col] = aggregated_by_portfolio_formatted[col].astype(str)
            
            aggregated_by_portfolio_formatted.to_excel(writer, sheet_name="Performance Agrégée", index=False)
            
            # Ajout des graphiques 
            performances.add_charts_by_profile_to_excel(writer, df_total)
            
            workbook  = writer.book
            header_format = workbook.add_format({
                'bold': True,
                'text_wrap': True,
                'valign': 'top',
                'fg_color': '#D7E4BC',
                'border': 1
            })
            
            # Mise en forme pour "Performances Hebdomadaires"
            worksheet_ph = writer.sheets["Performances Hebdomadaires"]
            df_ph_no_index = df_total_formatted.reset_index()
            for col_num, value in enumerate(df_ph_no_index.columns.values):
                worksheet_ph.write(0, col_num, value, header_format)
            for i, col in enumerate(df_ph_no_index.columns):
                max_len = max(df_ph_no_index[col].astype(str).map(len).max(), len(str(col))) + 2
                worksheet_ph.set_column(i, i, max_len)
            
            # Mise en forme pour "Performance Agrégée"
            worksheet_pa = writer.sheets["Performance Agrégée"]
            for col_num, value in enumerate(aggregated_by_portfolio_formatted.columns.values):
                worksheet_pa.write(0, col_num, value, header_format)
            for i, col in enumerate(aggregated_by_portfolio_formatted.columns):
                max_len = max(aggregated_by_portfolio_formatted[col].astype(str).map(len).max(), len(str(col))) + 2
                worksheet_pa.set_column(i, i, max_len)

            
        print("\nRapport Excel généré avec trois feuilles :")
        print("1) 'Performances Hebdomadaires'")
        print("2) 'Performance Agrégée' (agrégation par portefeuille)")
        print("3) 'Charts' (graphique(s) ajoutés)")
    else:
        print("Aucune performance calculée.")

if __name__ == '__main__':
    main()

=== Initialisation des bases de données ===
Bases de données initialisées.

[1/177] Importation de AAPL
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


[2/177] Importation de MSFT


[*********************100%***********************]  1 of 1 completed


[3/177] Importation de GOOGL


...


Semaine 103 : 2024-12-16 à 2024-12-23
→ Calcul des performances de 2024-12-09 à 2024-12-16


/Users/adam/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Equity Only: Prédictions: 100%|██████████| 115/115 [00:00<00:00, 502.65it/s]



Semaine 104 : 2024-12-23 à 2024-12-30
→ Calcul des performances de 2024-12-16 à 2024-12-23


/Users/adam/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Equity Only: Prédictions: 100%|██████████| 115/115 [00:00<00:00, 460.95it/s]


Calcul des performances de 2024-12-23 à 2024-12-30

Dernière période : Calcul des performances de 2024-12-23 à 2024-12-30

Rapport Excel généré avec trois feuilles :
1) 'Performances Hebdomadaires'
2) 'Performance Agrégée' (agrégation par portefeuille)
3) 'Charts' (graphique(s) ajoutés)
